In [1]:
import pandas as pd

In [9]:
solar = pd.read_csv('../data/transformed/solar.csv')
solar.loc[:, 'time_tag'] = pd.to_datetime(solar['time_tag']) 
solar.set_index('time_tag', inplace=True)
solar[['speed']]

c:\Users\Umair\Documents\Digital_Futures\Capstone\.venv\Lib\site-packages\pandas\core\indexes\base.py:7654: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


,speed
time_tag,
2025-11-24 14:11:00,721.3
2025-11-24 14:12:00,713.9
2025-11-24 14:13:00,645.6
2025-11-24 14:14:00,709.6
2025-11-24 14:15:00,716.7
...,...
2025-12-01 14:04:00,450.2
2025-12-01 14:05:00,448.5
2025-12-01 14:06:00,445.5


In [11]:
import matplotlib.pyplot as plt

In [10]:
solar.index

DatetimeIndex(['2025-11-24 14:11:00', '2025-11-24 14:12:00',
               '2025-11-24 14:13:00', '2025-11-24 14:14:00',
               '2025-11-24 14:15:00', '2025-11-24 14:16:00',
               '2025-11-24 14:17:00', '2025-11-24 14:18:00',
               '2025-11-24 14:19:00', '2025-11-24 14:20:00',
               ...
               '2025-12-01 13:59:00', '2025-12-01 14:00:00',
               '2025-12-01 14:01:00', '2025-12-01 14:02:00',
               '2025-12-01 14:03:00', '2025-12-01 14:04:00',
               '2025-12-01 14:05:00', '2025-12-01 14:06:00',
               '2025-12-01 14:07:00', '2025-12-01 14:08:00'],
              dtype='datetime64[ns]', name='time_tag', length=10078, freq=None)

In [31]:
solar.index

DatetimeIndex(['2025-11-24 14:11:00', '2025-11-24 14:12:00',
               '2025-11-24 14:13:00', '2025-11-24 14:14:00',
               '2025-11-24 14:15:00', '2025-11-24 14:16:00',
               '2025-11-24 14:17:00', '2025-11-24 14:18:00',
               '2025-11-24 14:19:00', '2025-11-24 14:20:00',
               ...
               '2025-12-01 13:59:00', '2025-12-01 14:00:00',
               '2025-12-01 14:01:00', '2025-12-01 14:02:00',
               '2025-12-01 14:03:00', '2025-12-01 14:04:00',
               '2025-12-01 14:05:00', '2025-12-01 14:06:00',
               '2025-12-01 14:07:00', '2025-12-01 14:08:00'],
              dtype='datetime64[ns]', name='time_tag', length=10078, freq=None)

In [37]:
import pandas as pd
import numpy as np
import altair as alt

# --- Data Generation (for context) ---
date_rng = pd.date_range(start=solar.index[0], end=solar.index[-1], freq='H')
df = solar.reset_index()
df = df[['time_tag','speed']]
df['Time'] = df['time_tag']

# --- Setup Parameters ---
one_day_ms = 24 * 60 * 60 * 1000
one_hour_ms = 60 * 60 * 1000 # New step size for smoothness

min_time = df['Time'].min()
max_time = df['Time'].max()

# 1. Define the Slider Input
slider = alt.binding_range(
    min=min_time.value // 10**6,
    max=(max_time - pd.Timedelta(days=1)).value // 10**6,
    step=one_hour_ms, # MODIFIED: Smoother step (hourly)
    name='' # MODIFIED: No slider label
)

# 2. Define the Parameter
start_time_param = alt.param(
    name='startTime',
    value=min_time.value // 10**6,
    bind=slider
)

# Define the filter expressions
end_time_expr = f'datum.Time < time(startTime) + {one_day_ms}'
start_time_expr = 'datum.Time >= time(startTime)'

# --- Chart Creation ---
chart = alt.Chart(df).mark_line().encode(
    # X-axis remains static, letting the filter define the fixed 1-day view
    x=alt.X('Time:T', title='Time'), 
    y=alt.Y('Value:Q', title='Value'),
    tooltip=[alt.Tooltip('Time:T'), alt.Tooltip('Value:Q')]
).add_params(
    start_time_param
).transform_filter(
    start_time_expr
).transform_filter(
    end_time_expr
).properties(
    title='Interactive Daily Window Plot (Smooth Slider)'
)

chart

C:\Users\Umair\AppData\Local\Temp\ipykernel_215720\3869295602.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_rng = pd.date_range(start=solar.index[0], end=solar.index[-1], freq='H')


MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).

Try enabling the VegaFusion data transformer which raises this limit by pre-evaluating data
transformations in Python.
    >> import altair as alt
    >> alt.data_transformers.enable("vegafusion")

Or, see https://altair-viz.github.io/user_guide/large_datasets.html for additional information
on how to plot large datasets.

alt.Chart(...)